In [1]:
import geopandas as gpd

In [2]:
dict_layer_year = {'argentina-2016-encuestas': 2016,
                   'catastro-peru-2018-para-datos-abiertos-ficha_catastro_peru':  2018,
                   'colombia-derecho-a-bogota-2015-encuestas': 2015,
                   'guatemala-2016-encuestas': 2016,
                   'honduras-2018-censo-de-asentamientos-informales': 2018,
                   'Limite_Favelas_2016': 2016,
                   'paraguay-2015-encuestas': 2016
}

dict_layer_country = {'argentina-2016-encuestas': 'arg',
                   'catastro-peru-2018-para-datos-abiertos-ficha_catastro_peru':  'per',
                   'colombia-derecho-a-bogota-2015-encuestas': 'col',
                   'guatemala-2016-encuestas': 'gua',
                   'honduras-2018-censo-de-asentamientos-informales': 'hon',
                   'Limite_Favelas_2016': 'bra',
                   'paraguay-2015-encuestas': 'par'
}

In [3]:
gpd.read_file('../data/informal_settlements_all.gpkg')

,layer,path,geometry
0,argentina-2016-encuestas,file:///Users/federico/Documents/dymaxion_labs...,"MULTIPOLYGON (((-58.80922 -34.53435, -58.80286..."
1,argentina-2016-encuestas,file:///Users/federico/Documents/dymaxion_labs...,"MULTIPOLYGON (((-68.09600 -38.94102, -68.09661..."
2,argentina-2016-encuestas,file:///Users/federico/Documents/dymaxion_labs...,"MULTIPOLYGON (((-68.11663 -38.92904, -68.11651..."
3,argentina-2016-encuestas,file:///Users/federico/Documents/dymaxion_labs...,"MULTIPOLYGON (((-68.11655 -38.93412, -68.11710..."
4,argentina-2016-encuestas,file:///Users/federico/Documents/dymaxion_labs...,"MULTIPOLYGON (((-68.07649 -38.94141, -68.07593..."
...,...,...,...
6639,paraguay-2015-encuestas,file:///Users/federico/Documents/dymaxion_labs...,"MULTIPOLYGON (((-57.54809 -25.20627, -57.54784..."
6640,paraguay-2015-encuestas,file:///Users/federico/Documents/dymaxion_labs...,"MULTIPOLYGON (((-57.52596 -25.23694, -57.52575..."
6641,paraguay-2015-encuestas,file:///Users/federico/Documents/dymaxion_labs...,"MULTIPOLYGON (((-57.42215 -25.23246, -57.42205..."
6642,paraguay-2015-encuestas,file:///Users/federico/Documents/dymaxion_labs...,GEOMETRYCOLLECTION EMPTY


In [4]:
is_path = '../data/informal_settlements_all.gpkg'
gdf = gpd.read_file(is_path)
gdf = gdf[~gdf.geometry.is_empty]
gdf.shape

(6446, 3)

In [5]:
gdf['class'] = 'A'

In [6]:
gdf.to_file('../data/informal_settlements_epsg4326_claseA.gpkg')

/Users/federico/opt/anaconda3/envs/satprocenv/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [7]:
gdf['geometry'] = gdf.to_crs('epsg:3857').buffer(2000).to_crs('epsg:4326')
gdf_dissolved = gdf.dissolve().explode().reset_index(drop=True)[['geometry']]
gdf_dissolved = gdf_dissolved.sjoin(gdf, how='left')
gdf_dissolved = gdf_dissolved.drop('index_right', axis=1).drop_duplicates()
gdf_dissolved.head()

/var/folders/nc/n4mcj2vn7pg5xhhqgdxkjcdh0000gn/T/ipykernel_13292/1754581195.py:2: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  gdf_dissolved = gdf.dissolve().explode().reset_index(drop=True)[['geometry']]


,geometry,layer,path,class
0,"POLYGON ((-90.52242 14.47005, -90.52356 14.468...",guatemala-2016-encuestas,file:///Users/federico/Documents/dymaxion_labs...,A
1,"POLYGON ((-90.59283 14.67266, -90.59349 14.674...",guatemala-2016-encuestas,file:///Users/federico/Documents/dymaxion_labs...,A
2,"POLYGON ((-76.96437 -12.14876, -76.96581 -12.1...",catastro-peru-2018-para-datos-abiertos-ficha_c...,file:///Users/federico/Documents/dymaxion_labs...,A
3,"POLYGON ((-74.17127 4.51761, -74.17198 4.51879...",colombia-derecho-a-bogota-2015-encuestas,file:///Users/federico/Documents/dymaxion_labs...,A
4,"POLYGON ((-74.21967 4.63494, -74.21797 4.63534...",colombia-derecho-a-bogota-2015-encuestas,file:///Users/federico/Documents/dymaxion_labs...,A


In [8]:
gdf_dissolved['year'] = gdf_dissolved['layer'].map(dict_layer_year)
gdf_dissolved['country'] = gdf_dissolved['layer'].map(dict_layer_country)
gdf_dissolved['id'] = gdf_dissolved.groupby('country').cumcount()+1

In [9]:
gdf_dissolved['class'] = 'A'

In [10]:
gdf_dissolved.shape

(365, 7)

In [11]:
gdf_dissolved.tail()

,geometry,layer,path,class,year,country,id
360,"POLYGON ((-54.59626 -25.89846, -54.59453 -25.8...",argentina-2016-encuestas,file:///Users/federico/Documents/dymaxion_labs...,A,2016,arg,352
361,"POLYGON ((-53.99624 -25.67202, -53.99586 -25.6...",argentina-2016-encuestas,file:///Users/federico/Documents/dymaxion_labs...,A,2016,arg,353
362,"POLYGON ((-54.52725 -25.63598, -54.52636 -25.6...",argentina-2016-encuestas,file:///Users/federico/Documents/dymaxion_labs...,A,2016,arg,354
363,"POLYGON ((-43.70421 -22.99074, -43.70433 -22.9...",Limite_Favelas_2016,/Users/federico/Downloads/Limite_Favelas_2016....,A,2016,bra,1
364,"POLYGON ((-43.09631 -22.78161, -43.09766 -22.7...",Limite_Favelas_2016,/Users/federico/Downloads/Limite_Favelas_2016....,A,2016,bra,2


In [12]:
gdf_dissolved.to_file('../data/informal_settlements_epsg4326_claseA_buff2000_dissolve.gpkg')

/Users/federico/opt/anaconda3/envs/satprocenv/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
